In [1]:
import scipy.sparse as sp
import numpy as np
dataset_name='ml-100k'
dataset_folder='ml'
preflix_folder='24_07_19'
#A=sp.load_npz(dataset_name+'/'+dataset_name+'_s_pre_adj_mat.npz')
A = sp.load_npz('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_pre_adj_mat_index_from_0.npz')#D^-1/2*A*D^-1/2
print(A.shape)

(2466, 2466)


In [3]:
item_num=1523
user_num=943
print(user_num,item_num)

943 1523


In [4]:
A_tmp=A[:user_num,user_num:]
print(A_tmp.shape)

(943, 1523)


In [5]:
np.sum(A_tmp.todense()!=0)

58196

In [6]:
num_hop = 3
A_tmp = A[:user_num, user_num:]
A_final = A[:user_num, user_num:]

for i in range(num_hop-1):
    print(i)
    if i % 2 == 0:
        A_tmp = A_tmp.dot(A[:user_num, user_num:].transpose())
    else:
        A_tmp = A_tmp.dot(A[:user_num, user_num:])
        A_final = A_tmp
    print(A_final.shape)

0
(943, 1523)
1
(943, 1523)


In [7]:
import numpy as np
A_bool=sp.csr_matrix(A_final).astype(bool)

In [8]:
unreachable=sp.csr_matrix(~A_bool.todense()).astype(int)

In [9]:
print(unreachable.getnnz())
print(unreachable.shape)

10954
(943, 1523)


In [10]:
def softmax_non_zero_elements(row):
    non_zero_elements = tf.boolean_mask(row, tf.not_equal(row, 0.0))
    softmax_non_zero = tf.nn.softmax(non_zero_elements)
    softmax_non_zero /= tf.reduce_sum(softmax_non_zero)
    # 创建一个与 row 形状相同的张量，填充 softmax 结果
    indices = tf.where(tf.not_equal(row, 0.0))
    indices = tf.cast(indices, tf.int64)  # 将索引转换为 int64 类型
    softmax_non_zero = tf.cast(softmax_non_zero, tf.float64)  # 确保 softmax_non_zero 是 float32 类型
    softmax_full_row = tf.scatter_nd(indices, softmax_non_zero, tf.cast(tf.shape(row), tf.int64)) 
    return softmax_full_row

In [12]:
#with probability
import tensorflow as tf
degree_weight=np.tile(np.array(A[:user_num,user_num:].todense().sum(0))[0],(user_num,1))
d_1=np.asarray(unreachable.todense())
p=degree_weight*d_1
print(p)
print(np.shape(p))
print(np.shape(np.where(p!=0)))
matrix = tf.constant(p,dtype=tf.float64)
non_zero_mask = tf.not_equal(matrix, 0.0)
non_zero_elements = tf.boolean_mask(matrix, non_zero_mask)


# 将softmax后的值放回原向量的位置
p = tf.map_fn(softmax_non_zero_elements, matrix)

#p=tf.nn.softmax(p)*d_1
a=[]
for i in range(user_num):
    if np.isclose(p[i].numpy().sum(), 1.0):
        prob=(p[i].numpy())/ (p[i].numpy().sum())
        a.append(np.random.choice(range(item_num), size=50,replace=True,p=prob).tolist())
    else:
        #print(i,p[i].numpy().sum())
        a.append([])
d_2=np.zeros_like(d_1)
for idx,i in enumerate(a):
    for j in i:
        d_2[idx,j]=1
#a=a%item_num
d_2=sp.csr_matrix(d_2)
print(d_2.getnnz())
print(d_2.shape)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.14285715 0.        ]]
(943, 1523)
(2, 10954)
9062
(943, 1523)


In [13]:
R = d_2.tocoo()
l_t_zero = sp.coo_matrix((R.shape[0],R.shape[0]), dtype=np.float32)
r_b_zero = sp.coo_matrix((R.shape[1],R.shape[1]), dtype=np.float32)
r_t_R=R
l_b_R=R.T
t_adj_mat=sp.hstack([l_t_zero,r_t_R])
b_adj_mat=sp.hstack([l_b_R,r_b_zero])
adj_mat=sp.vstack([t_adj_mat,b_adj_mat])
print('already create adjacency matrix', adj_mat.shape)

already create adjacency matrix (2466, 2466)


In [14]:
import numpy as np
adj_mat=adj_mat
rowsum = np.array(adj_mat.sum(1))
d_inv = np.power(rowsum, -0.5).flatten()
d_inv[np.isinf(d_inv)] = 0.
d_inv[np.isnan(d_inv)] = 0.
d_mat_inv = sp.diags(d_inv)
norm_adj = d_mat_inv.dot(adj_mat)
norm_adj = norm_adj.dot(d_mat_inv)
print('generate pre adjacency matrix.')
L_norm = sp.csr_matrix(sp.eye(user_num+item_num))-norm_adj
L_norm=L_norm.tocsr()
sp.save_npz('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_l_norm_3_prob_mat.npz', L_norm)

generate pre adjacency matrix.


/tmp/ipykernel_17958/3923550317.py:4: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()


In [ ]:
#topk
import tensorflow as tf
degree_weight=np.tile(np.array(A[:user_num,user_num:].todense().sum(0))[0],(user_num,1))
d_1=np.asarray(unreachable.todense())
p=degree_weight*d_1
need_mask_list=np.argsort(p)[:,-50:][0].tolist()
mask=np.zeros_like(p)
for i in range(p.shape[1]):
    if i in need_mask_list:
        mask[:,i]=1
p=mask*p
print(np.shape(p))
print(np.shape(np.where(p!=0)))
d_2=sp.csr_matrix(mask)

In [ ]:
R = d_2.tocoo()
l_t_zero = sp.coo_matrix((R.shape[0],R.shape[0]), dtype=np.float32)
r_b_zero = sp.coo_matrix((R.shape[1],R.shape[1]), dtype=np.float32)
r_t_R=R
l_b_R=R.T
t_adj_mat=sp.hstack([l_t_zero,r_t_R])
b_adj_mat=sp.hstack([l_b_R,r_b_zero])
adj_mat=sp.vstack([t_adj_mat,b_adj_mat])
print('already create adjacency matrix', adj_mat.shape)

In [ ]:
import numpy as np
adj_mat=adj_mat
rowsum = np.array(adj_mat.sum(1))
d_inv = np.power(rowsum, -0.5).flatten()
d_inv[np.isinf(d_inv)] = 0.
d_inv[np.isnan(d_inv)] = 0.
d_mat_inv = sp.diags(d_inv)
norm_adj = d_mat_inv.dot(adj_mat)
norm_adj = norm_adj.dot(d_mat_inv)
print('generate pre adjacency matrix.')
L_norm = sp.csr_matrix(sp.eye(user_num+item_num))-norm_adj
L_norm=L_norm.tocsr()
sp.save_npz('/root/autodl-nas/workspace/datasets/ml/'+dataset_name+'/'+date+'_gcn/' +dataset_name+'_s_l_norm_'+str(num_hop)+'_topk_mat.npz', L_norm)

In [135]:
print(sp.load_npz('E:/datasets/ml/ml-100k/'+date+'_gcn/' + dataset_name+'_s_l_norm_top_mat.npz'))#I-D^-1/2*A*D^-1/2

  (0, 0)	1.0
  (0, 1187)	-0.024999999999999998
  (0, 1280)	-0.06454972243679027
  (0, 1335)	-0.024999999999999998
  (0, 1489)	-0.024999999999999998
  (0, 1525)	-0.024999999999999998
  (0, 1595)	-0.024999999999999998
  (0, 1596)	-0.024999999999999998
  (0, 1653)	-0.024999999999999998
  (0, 1721)	-0.024999999999999998
  (0, 1727)	-0.024999999999999998
  (0, 1806)	-0.07905694150420949
  (0, 1824)	-0.024999999999999998
  (0, 1826)	-0.024999999999999998
  (0, 1829)	-0.05590169943749474
  (0, 1887)	-0.024999999999999998
  (0, 1894)	-0.024999999999999998
  (0, 1907)	-0.024999999999999998
  (0, 1911)	-0.024999999999999998
  (0, 1913)	-0.024999999999999998
  (0, 1936)	-0.024999999999999998
  (0, 1986)	-0.024999999999999998
  (0, 1989)	-0.024999999999999998
  (0, 2010)	-0.024999999999999998
  (0, 2012)	-0.07905694150420949
  :	:
  (2371, 15)	-0.02867696673382022
  (2371, 16)	-0.023918243661746996
  (2371, 17)	-0.028239124736245253
  (2371, 18)	-0.02366242621401505
  (2371, 2371)	1.0
  (2372, 0)	

In [51]:
print(sp.load_npz('E:/datasets/ml/ml-100k/'+date+'_gcn/' + dataset_name+'_s_l_norm_sample_mat.npz'))#I-D^-1/2*A*D^-1/2

  (0, 0)	1.0
  (0, 1187)	-0.024999999999999998
  (0, 1280)	-0.06454972243679027
  (0, 1335)	-0.024999999999999998
  (0, 1489)	-0.024999999999999998
  (0, 1525)	-0.024999999999999998
  (0, 1595)	-0.024999999999999998
  (0, 1596)	-0.024999999999999998
  (0, 1653)	-0.024999999999999998
  (0, 1721)	-0.024999999999999998
  (0, 1727)	-0.024999999999999998
  (0, 1806)	-0.07905694150420949
  (0, 1824)	-0.024999999999999998
  (0, 1826)	-0.024999999999999998
  (0, 1829)	-0.05590169943749474
  (0, 1887)	-0.024999999999999998
  (0, 1894)	-0.024999999999999998
  (0, 1907)	-0.024999999999999998
  (0, 1911)	-0.024999999999999998
  (0, 1913)	-0.024999999999999998
  (0, 1936)	-0.024999999999999998
  (0, 1986)	-0.024999999999999998
  (0, 1989)	-0.024999999999999998
  (0, 2010)	-0.024999999999999998
  (0, 2012)	-0.07905694150420949
  :	:
  (2371, 15)	-0.02867696673382022
  (2371, 16)	-0.023918243661746996
  (2371, 17)	-0.028239124736245253
  (2371, 18)	-0.02366242621401505
  (2371, 2371)	1.0
  (2372, 0)	

SyntaxError: unexpected EOF while parsing (1616222797.py, line 1)